<center><h1>Uma análise acerca do contexto político na cidade do Rio de Janeiro</h1>

---

<center><h6>Autor: Ronald Albert</h6>
<h6>Junho, 2021</h6>

<b>ATENÇÃO</b><br>
Este notebook demora em torno de 10  a 15 minutos para ser completamente executado. Recomendável deixar-lo executando enquanto passa um café ou algo parecido.

<b>Este trabalho utiliza os seguintes datasets:</b>
<ul>
<li>br_tse_filiacao_partidaria.filiacao_partidaria</li>
<li>br_tse_eleicoes.local_secao</li>
<li>br_tse_eleicoes.resultados_partido_secao</li>
</ul> 

Do datalake da basedosdados, disponível na [BigQuery](https://basedosdados.github.io/mais/access_data_bq/ "Base dos dados API") deles

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import basedosdados as bd
import re
import json
import string
import plotly
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
import scipy.stats as stats
import sympy as sym
import itertools
import os
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

Abaixo carregamos alguns csv's para auxiliarem a nossa análise:

---

Usamos o <b>'municipios.csv'</b> para conseguirmos o codigo do munícipio do Rio de Janeiro no IBGE, facilitando a consulta a base dos dados 

---

Usamos o <b>'orientacao_partidos.csv'</b> para termos a orientação política (esquerda, direita e centro) de cada partido político. <br>
Usaremos a classificação do [Congresso em Foco](https://congressoemfoco.uol.com.br/legislativo/direita-cresce-e-engole-o-centro-no-congresso-mais-fragmentado-da-historia) <br>
Esta tabela está disponível no [Wikipedia](https://pt.wikipedia.org/wiki/Lista_de_partidos_pol%C3%ADticos_do_Brasil)<br>

---

Outro "dataset" que usaremos é um [geojson](https://gist.github.com/esperanc/db213370dd176f8524ae6ba32433f90a) para nos auxiliar nas visualizações, assim como também nos ajudar a determinar em qual bairro estão localizadas cada uma das seções e zonas eleitorais.<br>
Publicado no github por <b>Claudio 'esperanc' Esperança</b>


In [ ]:
municipios = pd.read_csv("csv/municipios.csv")

orientacao_partidos = pd.read_csv("csv/orientacao_partidos.csv", error_bad_lines=False, delimiter=',')
orientacao_partidos =  orientacao_partidos.replace({ "—":"esquerda"})
orientacao_partidos['Nome'] = orientacao_partidos['Nome'].map(lambda x: x.upper())

with open('csv/bairros-geojson.json', encoding="utf8") as geojson:
    bairros = json.load(geojson)

Para não termos que fazer download dos dados todas as vezes que iniciarmos o notebook, faremos esse download uma única vez e armazenaremos seu resultado em um .csv.<br>

Para isso foi criado a função 'dataAlreadyBeenGot' que valida se o csv já foi populado com dados.<br>

<b>ATENÇÃO:</b><br>
Se este projeto foi clonado do github, os datasets já estão populados e não há necessidade de realizar query da BaseDosDados.<br> 
No entanto, se não se tem acesso à estes datasets, será necessário criar os arquivos .csv dentro de uma pasta nomeadas 'csv' no diretório do notebook. <br>
Os nomes dos arquivos csv são:
<ul>
<li>secao_eleitoral.csv</li>
<li>resultado_eleicoes.csv</li>
<li>filiados.csv</li>
</ul> 

In [ ]:
def dataAlreadyBeenGot(csvName):
    if os.stat(csvName).st_size == 0:
        return False
    else:
        return True

Abaixo carregamos o dataset de seções eleitorais da base dos dados:<br>
![alt text](img/secao_eleitoral.jpg)

Usaremos este dataset para determinar o bairro onde está localizado determinada zona eleitoral ou seção eleitoral. Para isso teremos a coluna 'melhor_urbano' que nos retornar a latitude e a longitude de tal seção e zona.

In [ ]:
if not dataAlreadyBeenGot('csv/secao_eleitoral.csv'):
    querySecaoEleitoral = """
        SELECT * FROM `basedosdados.br_tse_eleicoes.local_secao` AS LS
        WHERE LS.id_municipio={} AND LS.sigla_uf='RJ'
        """.format(municipios[municipios['nome_municipio'].eq('RIO DE JANEIRO')]['codigo_ibge'].values[0])

    secao_eleitoral = bd.read_sql(querySecaoEleitoral, billing_project_id='capacitacaoufrjanalytica')
    secao_eleitoral.to_csv("csv/secao_eleitoral.csv", index=False)
else:
    secao_eleitoral = pd.read_csv("csv/secao_eleitoral.csv")

Abaixo são filtradas as seções eleitorais onde as coordenadas não são informadas

In [ ]:
secao_eleitoral = secao_eleitoral[~(secao_eleitoral['melhor_urbano']).isnull()]

Abaixo carregamos o dataset de resultados por seção eleitoral da base dos dados:<br>
![alt text](img/resultados_partido.jpg)

Usaremos este dataset para construir nossas análises propriamente ditas, observando como os votos de determinado partido se comportam, dado ano, bairro e o cargo para o qual é a eleição

In [ ]:
if not dataAlreadyBeenGot('csv/resultado_eleicoes.csv'):
    queryResultadosPartidoMunicipio = """
        SELECT MZ.ano, MZ.zona, MZ.secao, MZ.sigla_partido, MZ.cargo, MZ.turno, MZ.votos_nominais, MZ.votos_nao_nominais FROM `basedosdados.br_tse_eleicoes.resultados_partido_secao` AS MZ
        WHERE MZ.id_municipio_tse={} AND MZ.sigla_uf='RJ'
        ORDER BY MZ.ano ASC""".format(municipios[municipios['nome_municipio'].eq('RIO DE JANEIRO')]['codigo_tse'].values[0])

    dfResultadosPartidoMunicipio = bd.read_sql(queryResultadosPartidoMunicipio, billing_project_id='capacitacaoufrjanalytica')

# Winding Number
Na célula seguinte, implementamos um algoritmo de rasterização de polígonos, chamado winding number<br>
O winding number funcionando determinando se algum ponto no plano cartesiano, está dentro ou não, de determinado polígono (também definido por pontos em um plano cartesiano) convexo ou não.<br>
O algoritmo será usado para determinar os bairros onde estão localizados as diferentes seções eleitorais do município do Rio de Janeiro. Dentro do geojson importado anteriormente, temos as coordenadas (latitude, longitude) de cada um dos pontos que definem cada bairro do município Rio de Janeiro.<br>
O seguinte algoritmo, será executado para cada uma das seções eleitorais e irá procurar dentro do geojson o primeiro bairro que, cujo qual o polígono contenha aquelas coordenadas.

Para mais informações sobre o winding number, visitar:<br>
[Geometry Algorithms](http://www.geomalgorithms.com/algorithms.html)

In [ ]:
def neighhboorhoodCoordinateIsInside(zona, secao):
    zona_secao_df = secao_eleitoral[secao_eleitoral['zona'].eq(zona) &  secao_eleitoral['secao'].eq(secao)]
    
    if len(zona_secao_df) == 0:
        return 'nao informado'
    
    coords = zona_secao_df.values[0][5]
    array_coords = [float(x) for x in re.findall(r'[-+]?\d+\.\d+', zona_secao_df.iloc[0]['melhor_urbano'])]
    
    longitude = array_coords[0]
    latitude = array_coords[1]
    for e in bairros['features']:
        if (e['geometry']['type'] != 'MultiPolygon'):
            vertices = e['geometry']['coordinates'][0]
        else:
            vertices = e['geometry']['coordinates'][0][0]
        wn = 0;
        i = 0
        j = len(vertices) - 1
        while (i < len(vertices)):
            v = vertices[j];
            v1 = vertices[i];
            if ( v[0] <= longitude ) :
                if ( longitude < v1[0] ):  
                    if ( (v1[1] - v[1]) * (longitude - v[0]) - (v1[0] - v[0]) * (latitude - v[1]) > 0 ):
                        wn += 1;
            else:
                if( longitude >= v1[0] ):
                    if ( (v1[1] - v[1]) * (longitude - v[0]) - (v1[0] - v[0]) * (latitude - v[1]) < 0 ):
                        wn -= 1;
            j = i
            i += 1
    
        if (wn != 0):
            return e['properties']['NOME']

Usando o algorítmo acima calculamos o bairro para cada combinação de zona e seção eleitoral possível no dataset, não fazemos isso no dataset diretamente, visto que os valores de zona e seção se repetem dentro dele, de forma que seria ineficaz calcular o bairro para cada linha do dataset.<br>
Assim calculamos o bairro para cada combinação de zona e seção eleitoral, e depois para associar o bairro a cada linha do dataset, basta realizar um merge usando a zona e seção eleitoral como chaves.

In [ ]:
if not dataAlreadyBeenGot('csv/resultado_eleicoes.csv'):
    returnBairro = np.vectorize(neighhboorhoodCoordinateIsInside)
    bairrosFromZonaSecao = pd.DataFrame(list(itertools.product(dfResultadosPartidoMunicipio['zona'].unique(),  dfResultadosPartidoMunicipio['secao'].unique())),
                                        columns=['zona', 'secao'])
    bairrosFromZonaSecao['bairro'] = returnBairro(bairrosFromZonaSecao['zona'].values, bairrosFromZonaSecao['secao'].values)

Abaixo realizamos o merge associando um bairro a cada linha do dataset<br>
Além disso, quando encontrarmos um NaN dentro da nossa coluna de bairros, temos que não tivemos resultado do bairro para aquela linha, dessa forma substituiremos todos os NaN's por 'nao informado'.

In [ ]:
if not dataAlreadyBeenGot('csv/resultado_eleicoes.csv'):
    dfResultadosPartidoMunicipio = pd.merge(dfResultadosPartidoMunicipio, bairrosFromZonaSecao, how='left', left_on=['secao', 'zona'], right_on = ['secao', 'zona'])
    dfResultadosPartidoMunicipio.replace(np.nan, 'nao informado', inplace=True)

Para associar uma orientação política a cada partido, precisamos tratar os nomes dos partidos presentes no nosso dataset para que estejam iguais aos presentes no csv 'orientacao_partidos.csv' <br>
Além disso, o BigQuery também apresenta partidos que mudaram de nome ou se fundiram com outros partidos com nomes diferentes, para esta análise trataremos todos os partidos com a sua sigla atual.
Por exemplo, o PRONA (Partido de Reedificação da Ordem Nacional) existiu até 2006 quando se filiou ao PL (Partido Liberal), para esta análise todos os votos de PRONA de 1998 até 2006 serão considerados como votos do PL

In [ ]:
if not dataAlreadyBeenGot('csv/resultado_eleicoes.csv'):
    dfResultadosPartidoMunicipio['sigla_partido'].replace({'PEN':'PATRIOTA', 'PATRI':'PATRIOTA', 
                                                           'PT DO B':'AVANTE', 'PT do B':'AVANTE', 
                                                           'PC do B': 'PCDOB', 'PC DO B': 'PCDOB',
                                                           'PRONA':'PL', 'PGT':'PL', 'PPB':'PP',
                                                           'PPS': 'CIDADANIA', 'PRN':'PTC',
                                                           'PRP': 'PATRIOTA', 'PSDC':'DC',
                                                           'PSN':'PODE', 'PST': 'PL', 'PTN':'PODE',
                                                           'PHS':'PODE', 'PRB':'REPUBLICANOS',
                                                           'PR': 'PL', 'PPL': 'PCDOB', 'SD': 'SOLIDARIEDADE',
                                                           'PAN':'PTB','PFL':'DEM', 'PMDB':'MDB'}, inplace=True)


Abaixo dissolvemos as colunas 'secao' e 'zona', visto que somente nos preocupamos com o bairro.<br>
Além disso também armazenamos o resultado do nosso tratamento em um csv, para que não haja necessidade desse tratamento ser reexecutado

In [ ]:
if not dataAlreadyBeenGot('csv/resultado_eleicoes.csv'):
    dfResultadosPartidoMunicipio = dfResultadosPartidoMunicipio.set_index(['ano', 'sigla_partido', 'bairro', 'cargo', 'turno']).groupby(level=(0, 1, 2, 3, 4)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()
    dfResultadosPartidoMunicipio.to_csv("csv/resultado_eleicoes.csv", index=False)
else:
    dfResultadosPartidoMunicipio = pd.read_csv("csv/resultado_eleicoes.csv")

Agruparemos os resultados somente por 'ano' e 'sigla_partido' para construir nossa primeira visualização <br>
Consideraremos somente os votos de primeiro turno, visto que, considerar os votos de ambos os turnos tornariam a análise enviesada em favor dos partidos que foram para o segundo turno. <br>
Também é criada uma coluna extra, chamada 'votos totais', que é a soma dos votos nominais com os votos não nominais.

In [ ]:
votosTotaisPorPartido = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'sigla_partido']).groupby(level=(0, 1)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()

votosTotaisPorPartido['votos totais'] = votosTotaisPorPartido['votos_nominais'] + votosTotaisPorPartido['votos_nao_nominais']

Para comparar o resultado na eleição de diferentes anos, é necessário considerar a quantidade de votos em cada um dos anos da eleição, visto que, determinado ano de eleição pode apresentar mais ou menos votantes do que outro.<br>

Dessa forma, criaremos outra coluna, denominada 'desempenho', onde nós temos a quantidade de voto de determinado partido dividido pela quantidade total de votos naquele ano.

In [ ]:
def returnPorcentagemDeVotos(partido, ano):
    votosPartido = votosTotaisPorPartido[votosTotaisPorPartido['sigla_partido'].eq(partido) & votosTotaisPorPartido['ano'].eq(ano)]['votos totais']
    votosTotaisBairro = votosTotaisPorPartido[votosTotaisPorPartido['ano'].eq(ano)]['votos totais'].sum()
    return votosPartido/votosTotaisBairro

porcentagemVetorizada = np.vectorize(returnPorcentagemDeVotos)
votosTotaisPorPartido['desempenho'] = porcentagemVetorizada(votosTotaisPorPartido['sigla_partido'], votosTotaisPorPartido['ano']) 

Segue o gráfico do desempenho de alguns partidos por ano.

<b>Note que para avaliar o desempenho de outros partidos, basta mudar as strings dentro do array 'partido'.</b> 

In [ ]:
partido = ['PT', 'PSDB', 'PSL']
fig = px.line(votosTotaisPorPartido[votosTotaisPorPartido['sigla_partido'].isin(partido)], x="ano", y="desempenho", 
              color='sigla_partido',
              title='Desempenho dos partidos '+str(partido)[1:-1]+' X ano')
fig.show()

Agora continuaremos a agrupar por 'ano' e 'sigla_partido', com a exceção de que adicionamos outra variável, sendo ela 'bairro'.<br>
Também calcularemos o desempenho de partidos por bairro, que vai ser calculado pela divisão do número de votos daquele partido em determinado ano e bairro pelo número total de votos naquele ano e bairro

In [ ]:
votosTotaisPartidoPorBairro = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'bairro','sigla_partido']).groupby(level=(0, 1, 2)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()

votosTotaisPartidoPorBairro['votos totais'] = votosTotaisPartidoPorBairro['votos_nominais'] + votosTotaisPartidoPorBairro['votos_nao_nominais']

def returnPorcentagemPorBairro(bairro, partido, ano):
    votosPartido = votosTotaisPartidoPorBairro[votosTotaisPartidoPorBairro['bairro'].eq(bairro) & votosTotaisPartidoPorBairro['sigla_partido'].eq(partido) & votosTotaisPartidoPorBairro['ano'].eq(ano)]['votos totais']
    votosTotaisBairro = votosTotaisPartidoPorBairro[votosTotaisPartidoPorBairro['bairro'].eq(bairro) & votosTotaisPartidoPorBairro['ano'].eq(ano)]['votos totais'].sum()
    return votosPartido/votosTotaisBairro

porcentagemPorBairroVetorizada = np.vectorize(returnPorcentagemPorBairro)

votosTotaisPartidoPorBairro['desempenho por bairro'] = porcentagemPorBairroVetorizada(votosTotaisPartidoPorBairro['bairro'], votosTotaisPartidoPorBairro['sigla_partido'], votosTotaisPartidoPorBairro['ano']) 

Construiremos um visualização do desempenho de partidos por bairro.

<b>Note que para avaliar o desempenho de outros partidos em outros bairro, basta mudar o valor das variáveis 'partido' e 'bairro'.</b> 

In [ ]:
partido = ['PT', 'PSL', 'PSOL', 'DEM']
bairro = 'Barra da Tijuca'
fig = px.line(votosTotaisPartidoPorBairro[votosTotaisPartidoPorBairro['sigla_partido'].isin(partido) & votosTotaisPartidoPorBairro['bairro'].eq(bairro)], 
              x="ano", y="desempenho por bairro", 
              color='sigla_partido',
              title='Desempenho dos partidos '+str(partido)[1:-1]+' no bairro '+bairro+' X ano')
fig.show()

Também podemos avaliar o desempenho de algum partido em diferentes bairros.

<b>Note que para avaliar o desempenho de outros partidos em outros bairro, basta mudar o valor das variáveis 'partido' e 'bairro'.</b> 

In [ ]:
partido = 'PSOL'
bairro = ['Bangu', 'Laranjeiras', 'Freguesia (Jacarepaguá)', 'Cidade Universitária']
fig = px.line(votosTotaisPartidoPorBairro[votosTotaisPartidoPorBairro['sigla_partido'].eq(partido) & votosTotaisPartidoPorBairro['bairro'].isin(bairro)], 
              x="ano", y="desempenho por bairro", 
              color='bairro',
              title='Desempenho do partido '+partido+' nos bairros '+str(bairro)[1:-1]+' X ano')
fig.show()

Como fizemos com o bairros, também podemos agrupar os resultados da eleições por cargo na eleição e avaliar os resultado para cargos específicos.<br>
São eles presidente, prefeito, governador, senador, vereador, deputado federal e deputado estadual

In [ ]:
votosTotaisPorCargoPorPartido = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano','sigla_partido', 'cargo']).groupby(level=(0, 1, 2)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()
votosTotaisPorCargoPorPartido['votos totais'] = votosTotaisPorCargoPorPartido['votos_nominais'] + votosTotaisPorCargoPorPartido['votos_nao_nominais']


Usaremos a mesma definição de desempenho de algum partido utilizada anteriormente: número de votos daquele partido para a eleição daquele cargo dividido pelo número de votos total naquela eleição para aquele cargo.

In [ ]:
def returnPorcentagemDeVotosParaCargo(partido, ano, cargo):
    votosPartido = votosTotaisPorCargoPorPartido[votosTotaisPorCargoPorPartido['sigla_partido'].eq(partido) & votosTotaisPorCargoPorPartido['ano'].eq(ano) & votosTotaisPorCargoPorPartido['cargo'].eq(cargo)]['votos totais']
    votosTotaisBairro = votosTotaisPorCargoPorPartido[votosTotaisPorCargoPorPartido['ano'].eq(ano) & votosTotaisPorCargoPorPartido['cargo'].eq(cargo)]['votos totais'].sum()
    return votosPartido/votosTotaisBairro

porcentagemPorCargoVetorizada = np.vectorize(returnPorcentagemDeVotosParaCargo)
votosTotaisPorCargoPorPartido['desempenho'] = porcentagemPorCargoVetorizada(votosTotaisPorCargoPorPartido['sigla_partido'], votosTotaisPorCargoPorPartido['ano'], votosTotaisPorCargoPorPartido['cargo']) 

Construiremos uma visualização do desempenho por cargo.

<b>Note que para avaliar o desempenho de outros partidos em eleições para outros cargos, basta mudar o valor das variáveis 'partido' e 'cargo'.</b> 

In [ ]:
cargo = 'prefeito'
partido = ['DEM', 'PSOL', 'REPUBLICANOS']
fig = px.line(votosTotaisPorCargoPorPartido[votosTotaisPorCargoPorPartido['sigla_partido'].isin(partido) & votosTotaisPorCargoPorPartido['cargo'].eq(cargo)], 
              x="ano", y="desempenho", 
              color='sigla_partido',
              title='Desempenho dos partidos '+str(partido)[1:-1]+' para o cargo de '+cargo+' X ano')
fig.show()

# Dividindo os partidos por orientação política

Abaixo definimos uma função para retornar a orientação política de determinado partido (esquerda, direita, centro).

In [ ]:
def returnOrientacaoPolitica(partido):
    return orientacao_partidos[orientacao_partidos['Nome'].eq(partido)]['Congresso em Foco (2019)[37]'].values[0]

Assim associamos cada partido com uma orientação política no nosso dataset

In [ ]:
dfResultadosPartidoMunicipio['orientacao politica'] = dfResultadosPartidoMunicipio['sigla_partido'].map(lambda x: returnOrientacaoPolitica(x))

Novamente, agruparemos os resultados das eleições por orientação política para depois avaliar o seu desempenho.
<br>
Utilizaremos a mesma noção de desempenho de anteriormente: votos de determinada orientação política em algum ano dividido pela quantidade total de votos naquele ano.

In [ ]:
votosPorOrientacaoPolitica = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'orientacao politica']).groupby(level=(0, 1)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()
votosPorOrientacaoPolitica['votos totais'] = votosPorOrientacaoPolitica['votos_nominais'] + votosPorOrientacaoPolitica['votos_nao_nominais']

In [ ]:
def returnPorcentagemDeVotosPorOrientacao(orientacao, ano):
    votosPartido = votosPorOrientacaoPolitica[votosPorOrientacaoPolitica['orientacao politica'].eq(orientacao) & votosPorOrientacaoPolitica['ano'].eq(ano)]['votos totais']
    votosTotaisBairro = votosPorOrientacaoPolitica[votosPorOrientacaoPolitica['ano'].eq(ano)]['votos totais'].sum()
    return votosPartido/votosTotaisBairro

porcentagemPorOrientacaoVetorizada = np.vectorize(returnPorcentagemDeVotosPorOrientacao)
votosPorOrientacaoPolitica['desempenho'] = porcentagemPorOrientacaoVetorizada(votosPorOrientacaoPolitica['orientacao politica'], votosPorOrientacaoPolitica['ano']) 

Assim também podemos montar uma visualização apresentado o desempenho por ano de cada uma das orientações políticas

In [ ]:
fig = px.line(votosPorOrientacaoPolitica, x="ano", y="desempenho", color='orientacao politica',
             title='Desempenho de determinada ideologica política X ano')
fig.show()

Analogamente aos exemplos anteriores, agruparemos os resultado das orientações políticas por bairro, e calcularemos o desempenho de cada orientação política por bairro.

In [ ]:
votosTotaisPartidoPorBairroPorOrientacao = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'bairro','orientacao politica']).groupby(level=(0, 1, 2)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()

votosTotaisPartidoPorBairroPorOrientacao['votos totais'] = votosTotaisPartidoPorBairroPorOrientacao['votos_nominais'] + votosTotaisPartidoPorBairroPorOrientacao['votos_nao_nominais']

def returnPorcentagemPorBairroPorOrientacao(bairro, orientacao, ano):
    votosPartido = votosTotaisPartidoPorBairroPorOrientacao[votosTotaisPartidoPorBairroPorOrientacao['bairro'].eq(bairro) & votosTotaisPartidoPorBairroPorOrientacao['orientacao politica'].eq(orientacao) & votosTotaisPartidoPorBairroPorOrientacao['ano'].eq(ano)]['votos totais']
    votosTotaisBairro = votosTotaisPartidoPorBairroPorOrientacao[votosTotaisPartidoPorBairroPorOrientacao['bairro'].eq(bairro) & votosTotaisPartidoPorBairroPorOrientacao['ano'].eq(ano)]['votos totais'].sum()
    return votosPartido/votosTotaisBairro

porcentagemPorBairroPorOriencaoVetorizada = np.vectorize(returnPorcentagemPorBairroPorOrientacao)

votosTotaisPartidoPorBairroPorOrientacao['desempenho por bairro'] = porcentagemPorBairroPorOriencaoVetorizada(votosTotaisPartidoPorBairroPorOrientacao['bairro'], votosTotaisPartidoPorBairroPorOrientacao['orientacao politica'], votosTotaisPartidoPorBairroPorOrientacao['ano']) 

Construiremos uma visualização do agrupamento de orientações políticas por bairro.


<b>Note que para avaliar o desempenho das orientações políticas em outros bairros, basta mudar o valor da variável 'bairro'.</b> 

In [ ]:
bairro = 'Méier'
fig = px.line(votosTotaisPartidoPorBairroPorOrientacao[votosTotaisPartidoPorBairroPorOrientacao['bairro'].eq(bairro)], 
              x="ano", y="desempenho por bairro", 
              color='orientacao politica',
              title='Desempenho de determinada orientação política no bairro '+bairro+' X ano')
fig.show()

Assim como para partidos específicos, podemos comparar o desempenho de alguma orientação política em diferentes bairros.

<b>Note que para avaliar o desempenho de outras orientações políticas em outros bairros, basta mudar o valor das variáveis 'orientacao' e 'bairro'.</b> 

In [ ]:
orientacao = 'esquerda'
bairro = ['Bangu', 'Laranjeiras', 'Freguesia (Jacarepaguá)']
fig = px.line(votosTotaisPartidoPorBairroPorOrientacao[votosTotaisPartidoPorBairroPorOrientacao['orientacao politica'].eq(orientacao) & votosTotaisPartidoPorBairroPorOrientacao['bairro'].isin(bairro)], 
              x="ano", y="desempenho por bairro", 
              color='bairro',
              title='Desempenho do(a) '+orientacao+' nos bairros '+str(bairro)[1:-1]+' X ano')
fig.show()

Agora, procuramos montar uma visualização de mapas, e dessa forma agruparemos por bairro, considerando somente o partido que teve mais votos para aquela eleição naquele bairro.

Assim podemos mostar quais partidos dominaram aquele bairro em determinado ano.

In [ ]:
auxDf = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)]

def calcularResultadoAgrupadoPorAno(ano):
    bairrosArray = []
    n_votosArray = []
    partidosArray = []

    for bairro, frame in auxDf[auxDf['ano'].eq(ano)].groupby('bairro'):
        n_votos = np.max(frame['votos_nominais'])
        partido = auxDf[auxDf['votos_nominais'].eq(n_votos) & auxDf['bairro'].eq(bairro) & auxDf['ano'].eq(ano)]['sigla_partido'].values[0]
        
        n_votosArray.append(n_votos)
        partidosArray.append(partido)
        bairrosArray.append(bairro)

    dfAgrupadoResultadosPartidoMunicipio = pd.DataFrame([bairrosArray, n_votosArray, partidosArray], 
                              index=['bairro', 'votos', 'partido']).T
    
    dfAgrupadoResultadosPartidoMunicipio['ano'] = [ano]*len(dfAgrupadoResultadosPartidoMunicipio)
    
    return dfAgrupadoResultadosPartidoMunicipio

dfAgrupadoResultadosPartidoMunicipioPorAno = pd.DataFrame()
for a in dfResultadosPartidoMunicipio['ano'].unique():
    dfAgrupadoResultadosPartidoMunicipioPorAno = dfAgrupadoResultadosPartidoMunicipioPorAno.append(calcularResultadoAgrupadoPorAno(a))
dfAgrupadoResultadosPartidoMunicipioPorAno.reset_index(drop=True, inplace=True)

Podemos montar uma visualização de mapas, usando o geojson importado anteriormente, para avaliar como se distribuem os partidos dado os bairros do Rio de Janeiro.<br>

<b>Note que para avaliar o mapa de eleições de outros anos basta mudar o valor da variável 'ano'.</b>

In [ ]:
ano = 2010
fig = px.choropleth_mapbox(dfAgrupadoResultadosPartidoMunicipioPorAno[dfAgrupadoResultadosPartidoMunicipioPorAno['ano'].eq(ano)], geojson=bairros, locations='bairro', color='partido',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['votos'],
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           title='Partido mais votado em cada bairro do Rio de Janeiro no ano '+str(ano),
                           zoom=9
                          )

fig.show()

Realizaremos o mesmo agrupamento para montar o mapa considerando as orientações políticas, para também montar uma visualização do mapa das orientações políticas no Rio de Janeiro.

In [ ]:
dfAgrupadoOrientacaoPoliticaMunicipioPorAno = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].eq(1)].set_index(['ano', 'orientacao politica', 'bairro']).groupby(level=(0, 1, 2)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()
    
dfAgrupadoOrientacaoPoliticaMunicipioPorAno['votos totais'] = dfAgrupadoOrientacaoPoliticaMunicipioPorAno['votos_nominais'] + dfAgrupadoOrientacaoPoliticaMunicipioPorAno['votos_nao_nominais']

dfOrientacaoPoliticaMunicipioMaxVotos = pd.DataFrame()
for group, frame in dfAgrupadoOrientacaoPoliticaMunicipioPorAno.set_index(['ano', 'bairro']).groupby(level = (0, 1)):
    n_votos = np.max(frame['votos totais'])
    dfOrientacaoPoliticaMunicipioMaxVotos = dfOrientacaoPoliticaMunicipioMaxVotos.append(frame[frame['votos totais'].eq(n_votos)])

dfOrientacaoPoliticaMunicipioMaxVotos.reset_index(inplace=True)

Construindo a visualização de mapas para diferentes orientações políticas<br>
<b>Note que para avaliar o mapa de eleições de outros anos basta mudar o valor da variável 'ano'.</b>

In [ ]:
ano = 2018
fig = px.choropleth_mapbox(dfOrientacaoPoliticaMunicipioMaxVotos[dfOrientacaoPoliticaMunicipioMaxVotos['ano'].eq(ano)], 
                           geojson=bairros, locations='bairro', color='orientacao politica',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['votos totais'],
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           title='Orientação política mais votada em cada bairro do Rio de Janeiro no ano '+str(ano),
                           zoom=9
                          )

fig.show()

# Filiação Partidária

As visualizações e códigos abaixo não dizem mais respeito a quantidade de votos em eleições, e sim aos filiados a algum partido.

Abaixo carregamos o dataset de Filiação Partidária da base dos dados:<br>
![alt text](img/filiados.jpg)

Usaremos este dataset para construir nossas análises propriamente ditas, observando como os filiados de determinado partido se comportam, dado ano, bairro e o cargo para o qual é a eleição.

Como não temos uma coluna do tabela no [BigQuery](https://basedosdados.github.io/mais/access_data_bq/ "Base dos dados API") que nos diz respeito a a quantidade de filiados por ano, realizaremos uma consulta para cada ano.<br>
Assim pegaremos o número de filiados que ainda são filiados e se filiaram antes daquele ano ou de ex-filiados que se desfiliaram depois daquele ano.

In [ ]:
if not dataAlreadyBeenGot('csv/filiados.csv'):
    dfFiliacaoPartidaria = pd.DataFrame()
    for ano in dfAgrupadoResultadosPartidoMunicipioPorAno['ano'].unique():
        queryFiliacaoPartidaria = """
            SELECT COUNT(*) AS n_filiados, FP.secao, FP.zona, FP.sigla_partido FROM `basedosdados.br_tse_filiacao_partidaria.filiacao_partidaria` AS FP
            WHERE FP.id_municipio_tse={} AND (FP.data_desfiliacao > '{}-12-30' OR (FP.situacao_registro = 'regular' AND FP.data_filiacao < '{}-01-01'))
            GROUP BY FP.zona, FP.sigla_partido, FP.secao
            ORDER BY FP.zona""".format(municipios[municipios['nome_municipio'].eq('RIO DE JANEIRO')]['codigo_tse'].values[0], ano, ano)

        dfResultadoQuery = bd.read_sql(queryFiliacaoPartidaria, billing_project_id='capacitacaoufrjanalytica')
        dfResultadoQuery['ano'] = [ano]*len(dfResultadoQuery)
        dfFiliacaoPartidaria = dfFiliacaoPartidaria.append(dfResultadoQuery)

Novamente cruzaremos o dados de zona e seção com o dataframe criado anteriormente para termos a quantidade de filiados por bairro.

In [ ]:
if not dataAlreadyBeenGot('csv/filiados.csv'):
    dfFiliacaoPartidaria = pd.merge(dfFiliacaoPartidaria, bairrosFromZonaSecao, how='left', left_on=['secao', 'zona'], right_on = ['secao', 'zona'])
    dfFiliacaoPartidaria.replace(np.nan, 'nao informado', inplace=True)


Assim, colapsaremos as colunas 'zona' e 'secao' e armazenaremos esse resultado em um csv para não termos que fazer download nem tratar os dados novamente

In [ ]:
if not dataAlreadyBeenGot('csv/filiados.csv'):
    dfFiliacaoPartidaria = dfFiliacaoPartidaria.set_index(['ano', 'sigla_partido', 'bairro']).groupby(level=(0, 1, 2)).agg({"n_filiados":np.sum}).reset_index()
    dfFiliacaoPartidaria.to_csv('csv/filiados.csv')
else:
    dfFiliacaoPartidaria = pd.read_csv("csv/filiados.csv")

Analogamente ao caso para desempenho dos partidos em eleições, construiremos uma visualização para o número de filiados por partido em determinado ano.

In [ ]:
filiadosPorPartido = dfFiliacaoPartidaria.set_index(['ano', 'sigla_partido']).groupby(level=(0, 1)).agg({"n_filiados":np.sum}).reset_index()

<b>Note que para avaliar o número de filiados de outros partidos, basta mudar o valor da variável 'partido'.</b> 

In [ ]:
partido = ['PSOL', 'DEM', 'REPUBLICANOS']
fig = px.line(filiadosPorPartido[filiadosPorPartido['sigla_partido'].isin(partido)], x="ano", y="n_filiados", 
              color = "sigla_partido",
              title='Filiados do '+str(partido)[1:-1]+' X ano')
fig.show()

Assim como realizado para outras visualizações também agruparemos a quantidade de filiados por partido em diferentes bairros do Rio de Janeiro.

In [ ]:
filiadosPartidoPorBairro = dfFiliacaoPartidaria.set_index(['ano', 'bairro','sigla_partido']).groupby(level=(0, 1, 2)).agg({"n_filiados":np.sum}).reset_index()

Construimos visualizações a partir desse agrupamento.

<b>Note que para avaliar o número de filiados de outros partidos em outros bairros, basta mudar os valores das variáveis 'partido' e 'bairro'.</b> 

In [ ]:
partido = ['PT', 'PSL', 'PSOL', 'DEM']
bairro = 'Barra da Tijuca'
fig = px.line(filiadosPartidoPorBairro[filiadosPartidoPorBairro['sigla_partido'].isin(partido) & filiadosPartidoPorBairro['bairro'].eq(bairro)], 
              x="ano", y="n_filiados", 
              color='sigla_partido',
              title='Filiados dos partido '+str(partido)[1:-1]+' no(a) '+bairro+' X ano')
fig.show()

In [ ]:
partido = 'PT'
bairro = ['Barra da Tijuca', 'Copacabana', 'Leblon']
fig = px.line(filiadosPartidoPorBairro[filiadosPartidoPorBairro['sigla_partido'].eq(partido) & filiadosPartidoPorBairro['bairro'].isin(bairro)], 
              x="ano", y="n_filiados", 
              color='bairro',
              title='Filiados dos partido '+partido+' no(a) '+str(bairro)[1:-1]+' X ano')
fig.show()

# Número de filiados por orientação política
Assim como no resultado em eleições, também avaliaremos o número de filiados por orientação política.<br>

Na seguinte célula, criamos uma nova coluna orientação política, onde temos a orientação política de cada partido em cada linha.

In [ ]:
dfFiliacaoPartidaria['orientacao politica'] = dfFiliacaoPartidaria['sigla_partido'].map(lambda x: returnOrientacaoPolitica(x))

Agruparemos o nosso dataframe por orientação política, para avaliar os filiados de cada orientação política por ano.

In [ ]:
filiadosPorOrientacaoPolitica = dfFiliacaoPartidaria.set_index(['ano', 'orientacao politica']).groupby(level=(0, 1)).agg({"n_filiados":np.sum}).reset_index()

Assim, construindo a seguinte visualização

In [ ]:
fig = px.line(filiadosPorOrientacaoPolitica, x="ano", y="n_filiados", color='orientacao politica',
             title='Filiados de determinada ideologica política X ano')
fig.show()

Também existe a possibilidade de agruparmos o número de filiados de cada orientação política por bairro.

In [ ]:
filiadosOrientacaoPorBairro = dfFiliacaoPartidaria.set_index(['ano', 'bairro','orientacao politica']).groupby(level=(0, 1, 2)).agg({"n_filiados":np.sum}).reset_index()

Construindo visualizações, a cerca desse agrupamento por bairro.

<b>Note que para avaliar o número de filiados de outros partidos em outros bairros, basta mudar os valores das variáveis 'orientacao' e 'bairro'.</b> 

In [ ]:
orientacao = 'esquerda'
bairro = ['Barra da Tijuca', 'Laranjeiras', 'Tanque', 'Copacabana']
fig = px.line(filiadosOrientacaoPorBairro[filiadosOrientacaoPorBairro['bairro'].isin(bairro) & filiadosOrientacaoPorBairro['orientacao politica'].eq(orientacao)], 
              x="ano", y="n_filiados", color='bairro',
             title='Filiados de partidos de '+orientacao+' nos(as) '+str(bairro)[1:-1]+' X ano')
fig.show()

In [ ]:
bairro = 'Méier'
fig = px.line(filiadosOrientacaoPorBairro[filiadosOrientacaoPorBairro['bairro'].eq(bairro)], 
              x="ano", y="n_filiados", color='orientacao politica',
             title='Filiados de determinada ideologica política no(a) '+bairro+' X ano')
fig.show()

Abaixo construimos o dataframe para montar a nossa visualização de mapas, agruparemos os resultado por bairro e por ano, tendo para cada valor de bairro e ano o partido com mais votos naquele ano e naquele bairro. 

In [ ]:
def calcularResultadoAgrupadoPorAno(ano):
    bairrosArray = []
    n_filiadosArray = []
    partidosArray = []
    
    for bairro, frame in dfFiliacaoPartidaria[dfFiliacaoPartidaria['ano'].eq(ano)].groupby('bairro'):
        n_filiados = np.max(frame['n_filiados'])
        partido = dfFiliacaoPartidaria[dfFiliacaoPartidaria['n_filiados'].eq(n_filiados) & dfFiliacaoPartidaria['bairro'].eq(bairro)]['sigla_partido'].values[0]
        
            
        n_filiadosArray.append(n_filiados)
        partidosArray.append(partido)
        bairrosArray.append(bairro)
        
    
    dfAgrupadoFiliacaoPartidaria = pd.DataFrame([bairrosArray, n_filiadosArray, partidosArray], 
                              index=['bairro', 'filiados', 'partido']).T
    
    dfAgrupadoFiliacaoPartidaria['ano'] = [ano]*len(dfAgrupadoFiliacaoPartidaria)
    
    return dfAgrupadoFiliacaoPartidaria

dfAgrupadoFiliacaoPartidariaPorAno = pd.DataFrame()
for a in dfFiliacaoPartidaria['ano'].unique():
    dfAgrupadoFiliacaoPartidariaPorAno = dfAgrupadoFiliacaoPartidariaPorAno.append(calcularResultadoAgrupadoPorAno(a))
dfAgrupadoFiliacaoPartidariaPorAno.reset_index(drop=True, inplace=True)                     

A partir desse dataframe, podemos construir uma visualização de mapas para o número de filiados no Rio de Janeiro.

<b>Note que para avaliar o mapa de filiados de outros anos basta mudar o valor da variável 'ano'.</b>

In [ ]:
ano = 2002
fig = px.choropleth_mapbox(dfAgrupadoFiliacaoPartidariaPorAno[dfAgrupadoFiliacaoPartidariaPorAno['ano'].eq(ano)], geojson=bairros, locations='bairro', color='partido',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['filiados'],
                           title='Partido com mais filiados em cada bairro do Rio de Janeiro no ano '+str(ano),
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

Analogamente ao mapa anterior, vamos agrupar por bairro e ano, no entanto, agora nos importaremos com as orientações políticas que receberam mais votos dado um ano e um bairro.

In [ ]:
dfAgrupadoFiliacaoOrientacaoPoliticaPorAno = dfFiliacaoPartidaria.set_index(['ano', 'orientacao politica', 'bairro']).groupby(level=(0, 1, 2)).agg({"n_filiados":np.sum}).reset_index()

dfOrientacaoPoliticaMunicipioMaxFiliados = pd.DataFrame()
for group, frame in dfAgrupadoFiliacaoOrientacaoPoliticaPorAno.set_index(['ano', 'bairro']).groupby(level = (0, 1)):
    n_filiados = np.max(frame['n_filiados'])
    dfOrientacaoPoliticaMunicipioMaxFiliados = dfOrientacaoPoliticaMunicipioMaxFiliados.append(frame[frame['n_filiados'].eq(n_filiados)])

dfOrientacaoPoliticaMunicipioMaxFiliados.reset_index(inplace=True)

A partir desse dataframe, podemos construir uma visualização de mapas para o número de filiados no Rio de Janeiro.

<b>Note que para avaliar o mapa de filiados de outros anos basta mudar o valor da variável 'ano'.</b>

In [ ]:
ano = 2020
fig = px.choropleth_mapbox(dfOrientacaoPoliticaMunicipioMaxFiliados[dfOrientacaoPoliticaMunicipioMaxFiliados['ano'].eq(ano)], 
                           geojson=bairros, locations='bairro', color='orientacao politica',
                           featureidkey="properties.NOME",
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           hover_data=['n_filiados'],
                           title='Orientação política com mais filiados em cada bairro do Rio de Janeiro no ano '+str(ano),
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()

# Correlação entre o Número de votos e Filiação Pártidaria

A partir desse pontos começamos a estudar as correlações entre a quantidade de filiados de determinado partido e a quantidade de votos que esse partido recebe em determinada eleição

Nas seguintes células iremos agrupar a quantidade de votos por ano, partido, cargo e bairo<br>
Assim como iremos agrupar a quantidade de filiados por ano, partido e bairro <br>

Juntaremos esses dataframes com a intenção de construir um grafico de distribuição dos votos pela filiação partidária na cidade do Rio de Janeiro.

In [ ]:
dfVotosAgrupados = dfResultadosPartidoMunicipio[dfResultadosPartidoMunicipio['turno'].ne(2)].set_index(['ano', 'sigla_partido', 'cargo', 'bairro']).groupby(level=(0, 1, 2, 3)).agg({"votos_nominais":np.sum, "votos_nao_nominais":np.sum}).reset_index()
dfVotosAgrupados.columns = ['ano', 'partido', 'cargo', 'bairro', 'votos_nominais', 'votos_nao_nominais']

dfVotosAgrupados['votos'] = dfVotosAgrupados['votos_nominais'].values + dfVotosAgrupados['votos_nao_nominais'].values

In [ ]:
dfFiliadosAgrupados = dfFiliacaoPartidaria.set_index(['ano', 'sigla_partido', 'bairro']).groupby(level=(0, 1, 2)).agg({"n_filiados":np.sum}).reset_index()
dfFiliadosAgrupados.columns = ['ano', 'partido', 'bairro', 'filiados']

In [ ]:
new_df = pd.merge(dfFiliadosAgrupados, dfVotosAgrupados,  how='inner', on=['ano', 'partido', 'bairro'])

Após juntar ambos os dataframes convertemos os valores de filiados e bairros que iremos usar para inteiro.

In [ ]:
new_df['filiados'] = new_df['filiados'].values.astype(int)
new_df['votos'] = new_df['votos'].values.astype(int)

Desconsideraremos a coluna cargos para construir a correlação, visto que, essa informação não faz sentido quando pensamos em filação partidária, assim não sendo possível usar essa informação na correlação entre filiação partidária e número de votos

In [ ]:
dfWithoutCargos = new_df.set_index(['ano', 'partido', 'bairro']).groupby(level=(0, 1, 2)).agg({"votos":np.sum, "filiados":np.max}).reset_index()

Realizamos uma regressão linear do número de filiados pelo número de votos para mostrar a reta que tenta explicar a relação entre os dois

In [ ]:
reg = LinearRegression().fit(dfWithoutCargos['filiados'].values.reshape(-1, 1),dfWithoutCargos['votos'].values )

Construimos uma visualização, sendo um gráfico de distriuição do número de filiados pelo número de votos, e uma reta que foi construida pela regressão linear, sendo a reta que melhor aproxima esse gráfico de distribuição.

In [ ]:
fig = px.scatter(dfWithoutCargos, x="filiados", y="votos", color='partido', hover_data=['ano', 'bairro'])

fig.add_trace(
    go.Scatter(
        x=dfWithoutCargos['filiados'].values,
        y=reg.predict(dfWithoutCargos['filiados'].values.reshape(-1, 1)),
        mode="lines",
        line=go.scatter.Line(color="gray"),
        showlegend=False)
)
fig.update_layout(
        title='Número de votos X Número de filiados',
)
fig.show()

Abaixo está a correlação entre ambas as variáveis

In [ ]:
dfWithoutCargos[['filiados', 'votos']].corr()

No entanto, para uma análise mais aprofundada, precisamos avaliar essa correlação em diferentes contextos.

Abaixo seguem os dataframes com as correlações entre filiação partidária e número de votos, dado determinado partido, bairro ou cargo de eleição

In [ ]:
dfPartidos = pd.DataFrame({'partido': dfWithoutCargos[:-1]['partido'].unique()})
dfPartidos['corr'] = dfPartidos['partido'].map(lambda x: stats.pearsonr(dfWithoutCargos[dfWithoutCargos['partido'].eq(x)]['filiados'], dfWithoutCargos[dfWithoutCargos['partido'].eq(x)]['votos'])[0])
dfPartidos.head()

In [ ]:
dfBairros = pd.DataFrame({'bairro': dfWithoutCargos['bairro'].unique()})
dfBairros['corr'] = dfBairros['bairro'].map(lambda x: stats.pearsonr(dfWithoutCargos[dfWithoutCargos['bairro'].eq(x)]['filiados'], dfWithoutCargos[dfWithoutCargos['bairro'].eq(x)]['votos'])[0])

dfBairros.head()

In [ ]:
dfCargos = pd.DataFrame({'cargo': new_df['cargo'].unique()})
dfCargos['corr'] = dfCargos['cargo'].map(lambda x: stats.pearsonr(new_df[new_df['cargo'].eq(x)]['filiados'], new_df[new_df['cargo'].eq(x)]['votos'])[0])

dfCargos.head()

Para analisar bem essa correlação dentro do contexto de cada partido especificamente, realizaremos uma regressão linear dessas duas variáveis (Filiação partidária e Número de votos) para cada partido, para conseguirmos construir uma visualização do gráfico de distribuição com a reta que melhor aproxima tal gráfico

In [ ]:
regDictPartido = {}
for partido in dfWithoutCargos['partido'].unique():
    regDictPartido[partido] = LinearRegression().fit(dfWithoutCargos[dfWithoutCargos['partido'].eq(partido)]['filiados'].values.reshape(-1, 1),dfWithoutCargos[dfWithoutCargos['partido'].eq(partido)]['votos'].values )

Abaixo, construimos a visualização com o gráfico de distribuição do Número de votos X Número de filiados do partido, e também mostramos a reta que melhor aproxima os pontos no gráfico de distribuição.

<b>Note que para mostrar o gráfico de distribuição de outro partido, basta mudar o valor da variável 'partido'</b>

In [ ]:
partido = 'NOVO'
fig = px.scatter(dfWithoutCargos[dfWithoutCargos['partido'].eq(partido)], x="filiados", y="votos", color='bairro', hover_data=['ano', 'bairro'])

fig.add_trace(
    go.Scatter(
        x=dfWithoutCargos[dfWithoutCargos['partido'].eq(partido)]['filiados'].values,
        y=regDictPartido[partido].predict(dfWithoutCargos[dfWithoutCargos['partido'].eq(partido)]['filiados'].values.reshape(-1, 1)),
        mode="lines",
        line=go.scatter.Line(color="gray"),
        showlegend=False)
)

fig.update_layout(
        title='Número de votos X Número de filiados do partido '+partido,
)
fig.show()

Analogamente para os bairros, realizaremos uma regressão linear dessas duas variáveis (Filiação partidária e Número de votos) para cada bairro, para conseguirmos construir uma visualização do gráfico de distribuição com a reta que melhor aproxima tal gráfico

In [ ]:
regDictBairro = {}

for bairro in dfWithoutCargos['bairro'].unique():
    regDictBairro[bairro] = LinearRegression().fit(dfWithoutCargos[dfWithoutCargos['bairro'].eq(bairro)]['filiados'].values.reshape(-1, 1),dfWithoutCargos[dfWithoutCargos['bairro'].eq(bairro)]['votos'].values )

Abaixo, construimos a visualização com o gráfico de distribuição do Número de votos X Número de filiados em determinado bairro, e também mostramos a reta que melhor aproxima os pontos no gráfico de distribuição

<b>Note que para mostrar o gráfico de distribuição em outro bairro, basta mudar o valor da variável 'bairro'</b>

In [ ]:
bairro = 'Cidade Universitária'
fig = px.scatter(dfWithoutCargos[dfWithoutCargos['bairro'].eq(bairro)], x="filiados", y="votos", color='partido', hover_data=['ano', 'bairro'])

fig.add_trace(
    go.Scatter(
        x=dfWithoutCargos[dfWithoutCargos['bairro'].eq(bairro)]['filiados'].values,
        y=regDictBairro[bairro].predict(dfWithoutCargos[dfWithoutCargos['bairro'].eq(bairro)]['filiados'].values.reshape(-1, 1)),
        mode="lines",
        line=go.scatter.Line(color="gray"),
        showlegend=False)
)
fig.update_layout(
        title='número de votos X número de filiados em '+bairro,
)
fig.show()

# Avaliando quais características dos partidos influenciam mais para uma maior correlação entre Filiação Partidária e Número de votos.

Temos em mãos somente uma informação referente aos partidos, que é a 'orientação política', dessa forma, vamos avaliar o quanto tal característica influencia na correlação entre Filiação Partidária e Número de Votos.

In [ ]:
dfPartidos['orientacao politica'] = dfPartidos['partido'].map(lambda x: orientacao_partidos[orientacao_partidos['Nome'].eq(x)]['Congresso em Foco (2019)[37]'].iloc[0])

Abaixo definimos as funções para cálculo da Entropia (calcularEntropia) e Informação Mútua (calcularGanho), que irão nos auxiliar a determinar a relação de dependência entre a orientação política e a correlação

In [ ]:
def calcularEntropia(df, coluna):
    values = df[coluna].unique()
    entropia = 0
    for i in values:
        pi = len(df[df[coluna].eq(i)])/len(df)
        entropia += pi*np.log2(pi)
    return -entropia

def calcularGanho(df, resultado, coluna):
    values = df[coluna].unique()
    ganho = 0
    for i in values:
        pi = len(df[df[coluna].eq(i)])/len(df)
        ganho += pi * calcularEntropia(df[df[coluna].eq(i)], resultado)
    ganho = calcularEntropia(df, resultado) - ganho
    return ganho

Para usarmos o cálculo de Informação Mútua, discretizaremos a coluna de correlação, usando as seguintes funções:

In [ ]:
def retornarValorDiscretizado(x, minValue, maxValue, bins):
    for i in range(1, bins + 1):
        if x <= minValue + i * ((maxValue - minValue)/bins):
            return str(round(minValue + (i-1) * ((maxValue - minValue)/bins), 2))+ " - " + str(round(minValue + i * ((maxValue - minValue)/bins), 2))

def discretizar(i):
    minValue = np.min(i)
    maxValue = np.max(i)
    bins = int(np.sqrt(len(i)))
            
    return i.map(lambda v:retornarValorDiscretizado(v, minValue, maxValue, bins))

dfPartidos['corr discretizada'] = discretizar(dfPartidos['corr'])

Abaixo mostramos o dataframe para mostrar o resultado da Informação mútua na coluna 'orientacao politica'

In [ ]:
partidosInformacaoMutua = pd.DataFrame()

partidosInformacaoMutua = partidosInformacaoMutua.append({'coluna': 'orientacao politica',
                                                        'informação mutua': calcularGanho(dfPartidos, 'corr discretizada', 'orientacao politica')},
                                                        ignore_index=True)

A visualização será um gráfico de barras com uma única barra, que indica a Informação Mútua da variável Orientação Política pela variável correlação discretizada.

In [ ]:
fig = px.bar(partidosInformacaoMutua, x='coluna', y='informação mutua', color='coluna',
            title='Informação Mútua por coluna dos Partidos')
fig.show()

Abaixo podemos ver a correlação média de cada um dos grupos para orientação política <br>


In [ ]:
partidosCorrMediaOrientacaoPolitica = dfPartidos.groupby('orientacao politica').agg({'corr': np.average}).reset_index()

fig = px.bar(partidosCorrMediaOrientacaoPolitica, x='orientacao politica', y='corr', color='orientacao politica',
            title='Correlação media por orientacao politica')
fig.show()

# Avaliando quais características dos bairros influenciam mais para uma maior correlação entre Filiação Partidária e Número de votos.

Realizaremos um processo análogo para as informações que termos sobre os bairros, com a exceção de que agora temos mais informações sobre cada um dos bairros.


Para isso usaremos a tabela presente em [Lista do bairros do Rio de Janeiro por IDH](https://pt.wikipedia.org/wiki/Lista_de_bairros_do_Rio_de_Janeiro_por_IDH)

Assim temos algumas variáveis sobre os bairros que podemos usar para explicar a correlação

Nas seguintes células realizaremos o tratamentos dos dados coletados da wikipedia para os cruzamos com os dataframe das correlações por bairros

In [ ]:
bairro_idh = pd.read_csv('csv/bairro_idh.csv')
bairro_idh = bairro_idh.iloc[1:]
bairro_idh.columns = ['N', 'bairro', 'exp de vida', 'alfabetizacao', 'frequencia escolar', 'renda per capita' ,'IL', 'IE', 'IR', 'IDH']
bairro_idh = bairro_idh.replace(to_replace ='\[\w\]', value = '', regex = True)
bairro_idh.replace(to_replace =',', value = '.', regex = True, inplace=True)
        
cols=[i for i in bairro_idh.columns if i not in ["N","bairro"]]
for col in cols:
    bairro_idh[col]=pd.to_numeric(bairro_idh[col])

In [ ]:
dfBairros = pd.merge(dfBairros, bairro_idh,  how='inner', left_on=['bairro'], right_on = ['bairro'])

Usaremos o arquivo 'bairro_ids.csv', para termos a Região de cada um dos bairros do Rio de Janeiro, tal tabela está presente em [Lista de bairros do Rio de Janeiro por Índice de Desenvolvimento Social](https://pt.wikipedia.org/wiki/Lista_de_bairros_do_Rio_de_Janeiro_por_%C3%8Dndice_de_Desenvolvimento_Social)

In [ ]:
bairro_ids = pd.read_csv('csv/bairro_ids.csv')
bairro_ids.columns = ['N', 'bairro', 'R A', 'regiao', 'IDS']
bairro_ids.replace(to_replace =',', value = '.', regex = True, inplace=True)
bairro_ids['IDS'] = pd.to_numeric(bairro_ids['IDS'])

In [ ]:
dfBairros = pd.merge(dfBairros, bairro_ids.loc[:, ~bairro_ids.columns.isin(['N'])],  how='inner', left_on=['bairro'], right_on = ['bairro'])

Assim, discretizamos as nossas colunas numéricas, para assim podermos usar o cálculo da Informação Mútua

In [ ]:
dfBairros['exp de vida discretizada'] = discretizar(dfBairros['exp de vida'])
dfBairros['alfabetizacao discretizada'] = discretizar(dfBairros['alfabetizacao'])
dfBairros['frequencia escolar discretizada'] = discretizar(dfBairros['frequencia escolar'])
dfBairros['renda per capita discretizada'] = discretizar(dfBairros['renda per capita'])
dfBairros['corr discretizada'] = discretizar(dfBairros['corr'])

Construimos um dataframe com os valores da Informação Mútua para cada uma das colunas discretizadas e da coluna 'regiao' pela correlação da Filiação Partidária pelo Número de votos. 

In [ ]:
bairrosInformacaoMutua = pd.DataFrame()

for c in ['exp de vida discretizada', 'alfabetizacao discretizada', 'frequencia escolar discretizada', 'renda per capita discretizada', 'regiao']:
    bairrosInformacaoMutua = bairrosInformacaoMutua.append({'coluna': c, 
                                                            'informação mutua': calcularGanho(dfBairros, 'corr discretizada', c)},
                                                           ignore_index=True)

Abaixo temos uma visualização em gráfico de barras para cada coluna com o valor da sua respectiva Informação Mútua

In [ ]:
fig = px.bar(bairrosInformacaoMutua, x='coluna', y='informação mutua', color='coluna',
            title='Informação Mútua por coluna dos Bairros')
fig.show()

Na seguintes células temos um gráficos de barras para cada coluna do dataframe de Bairros, cada um dos gráficos de barras mostra a correlação média entre Filiação partidária e o Número de filiados para todos os possíveis valores das colunas

In [ ]:
bairrosCorrMediaoRegiao = dfBairros.groupby('regiao').agg({'corr': np.average}).reset_index()

fig = px.bar(bairrosCorrMediaoRegiao, x='regiao', y='corr', color='regiao',
            title='Correlação media por região')
fig.show()

In [ ]:
bairrosCorrMediaExpDeVida = dfBairros.groupby('exp de vida discretizada').agg({'corr': np.average}).reset_index()

fig = px.bar(bairrosCorrMediaExpDeVida, x='exp de vida discretizada', y='corr', color='exp de vida discretizada',
            title='Correlação media por Expectativa de Vida')
fig.show()

In [ ]:
bairrosCorrMediaRenda = dfBairros.sort_values('renda per capita').groupby('renda per capita discretizada', sort=False).agg({'corr': np.average}).reset_index()

fig = px.bar(bairrosCorrMediaRenda, x='renda per capita discretizada', y='corr', color='renda per capita discretizada',
            title='Correlação media por Renda per Capita')
fig.show()

In [ ]:
bairrosCorrMediaAlfabetizacao = dfBairros.sort_values('alfabetizacao').groupby('alfabetizacao discretizada', sort=False).agg({'corr': np.average}).reset_index()


fig = px.bar(bairrosCorrMediaAlfabetizacao, x='alfabetizacao discretizada', y='corr', color='alfabetizacao discretizada',
            title='Correlação media por Alfabetização')
fig.show()

In [ ]:
bairrosCorrMediaFrequenciaEscolar = dfBairros.sort_values('frequencia escolar').groupby('frequencia escolar discretizada', sort=False).agg({'corr': np.average}).reset_index()


fig = px.bar(bairrosCorrMediaFrequenciaEscolar, x='frequencia escolar discretizada', y='corr', color='frequencia escolar discretizada',
            title='Correlação media por Frequência Escolar')
fig.show()

# Um modelo de clusterização para classificar bairros do Rio de Janeiro

Nas seguintes células, usaremos o algoritmo k-means para clusterizar os bairros do Rio de Janeiro, considerando o desempenho de orientações políticas naquele bairro, e o número de filiados de determinada orientação política em tal bairro do Rio de Janeiro.

A ideia por trás dessa clusterização é entender com mais precisão como os diferentes bairros do Rio de Janeiro se comportam em eleições.

Assim calcularemos a quantidade de votos que as orientações tiveram em cada bairro, assim como a quantidade de filiados que as orientações tem em cada bairro.<br>
Após isso, teremos mais 6 colunas em nosso dataframe 3 representado a quantidade de votos em cada uma das orientações políticas e outra 3 representando a quantidade de filiados de cada uma da orientações política para cada bairro.

Normalizaremos essas 6 variáveis, usando o método do sklearn 'StandardScaler', e realizaremos duas clusterizações, uma será para os votos de cada orientação política e outra para os filiados de cada orientação política.

In [ ]:
def desempenhoVotos(orientacao, bairro):
    votosPorBairroPorOrientacao = dfForKmeans[dfForKmeans['orientacao politica'].eq(orientacao) & dfForKmeans['bairro'].eq(bairro)]['votos']
    votosPorBairro = dfForKmeans[dfForKmeans['bairro'].eq(bairro)]['votos'].sum()
    
    return votosPorBairroPorOrientacao/votosPorBairro

def desempenhoFiliados(orientacao, bairro):
    filiadosPorBairroPorOrientacao = dfForKmeans[dfForKmeans['orientacao politica'].eq(orientacao) & dfForKmeans['bairro'].eq(bairro)]['filiados']
    filiadosPorBairro = dfForKmeans[dfForKmeans['bairro'].eq(bairro)]['filiados'].sum()
    
    return filiadosPorBairroPorOrientacao/filiadosPorBairro

desempenhoVotos = np.vectorize(desempenhoVotos)
desempenhoFiliados = np.vectorize(desempenhoFiliados)

dfForKmeans = dfWithoutCargos.copy()
dfForKmeans['orientacao politica'] = dfForKmeans['partido'].map(lambda x: returnOrientacaoPolitica(x))
dfForKmeans = dfForKmeans.set_index(['bairro', 'orientacao politica']).groupby(level=(0, 1)).agg({"votos":np.sum, "filiados":np.max}).reset_index()
dfForKmeans['desempenho votos'] = desempenhoVotos(dfForKmeans['orientacao politica'].values, dfForKmeans['bairro'].values)
dfForKmeans['desempenho filiados'] = desempenhoFiliados(dfForKmeans['orientacao politica'].values, dfForKmeans['bairro'].values)

dfForKmeans = pd.pivot_table(dfForKmeans, values=['desempenho votos', 'desempenho filiados'], index=['bairro'], columns=['orientacao politica'], aggfunc={np.mean})
dfForKmeans.reset_index(inplace=True)
dfForKmeans.columns = ['bairro', 'filiados_centro', 'filiados_direita', 'filiados_esquerda', 'votos_centro', 'votos_direita' , 'votos_esquerda']

dfForKmeans[['votos_esquerda', 'votos_direita', 'votos_centro', 'filiados_esquerda', 'filiados_direita', 'filiados_centro']] = StandardScaler().fit_transform(dfForKmeans[['votos_esquerda', 'votos_direita', 'votos_centro', 'filiados_esquerda', 'filiados_direita', 'filiados_centro']])

Abaixo realizamos a clusterização para cada bairro, com as variáveis número de votos da orientação política centro, número de votos da orientação política esquerda e número de votos da orientação política direita.

In [ ]:
kmeansVotos = KMeans(n_clusters=3).fit(dfForKmeans[dfForKmeans.columns.difference(['bairro', 'filiados_centro', 'filiados_direita', 'filiados_esquerda'])].values)
dfForKmeans['labels votos'] = kmeansVotos.labels_

Mostramos uma gráfico de distribuição 3d onde podemos ver essas classes se diferenciando

In [ ]:
fig = px.scatter_3d(dfForKmeans,
                 x="votos_esquerda", y="votos_direita", z='votos_centro',color='labels votos', hover_data=['bairro'])
fig.show()

Dessa forma, usaremos a clusterização para para colorir o mapa do Rio de Janeiro, dado as classificações geradas pelo K-means.

In [ ]:
fig = px.choropleth_mapbox(dfForKmeans, 
                           geojson=bairros, locations='bairro', color='labels votos',
                           featureidkey="properties.NOME",
                           mapbox_style="carto-positron",
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           title="Mapa do Rio de Janeiro colorido com a clusterização da variável filiados",
                           zoom=9
                          )

fig.show()

Abaixo realizamos a clusterização para cada bairro, com as variáveis número de filiados da orientação política centro, número de filiados da orientação política esquerda e número de filiados da orientação política direita.

In [ ]:
kmeansFiliados = KMeans(n_clusters=3).fit(dfForKmeans[dfForKmeans.columns.difference(['bairro', 'votos_centro', 'votos_direita' , 'votos_esquerda'])].values)
dfForKmeans['labels filiados'] = kmeansFiliados.labels_

Mostramos uma gráfico de distribuição 3d onde podemos ver os 4 clusters se diferenciando

In [ ]:
fig = px.scatter_3d(dfForKmeans,
                 x="filiados_esquerda", y="filiados_direita", z='filiados_centro', color='labels filiados', hover_data=['bairro'])
fig.show()

Dessa forma, usaremos a clusterização para para colorir o mapa do Rio de Janeiro, dado as classificações geradas pelo K-means.

In [ ]:
fig = px.choropleth_mapbox(dfForKmeans, 
                           geojson=bairros, locations='bairro', color='labels filiados',
                           featureidkey="properties.NOME",
                           mapbox_style="carto-positron",
                           center={"lon": -43.1811097599, "lat": -22.895417528999 },
                           zoom=9
                          )

fig.show()